In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
from time import sleep

driver = webdriver.Chrome()
driver.get('https://shopee.co.th/search?keyword=jewelry')
sleep(3)

thai_button = driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div[1]/div/div[3]/div[1]/button')
thai_button.click()

username = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[2]/div/div/div/div[2]/form/div/div[2]/div[2]/div[1]/input')
password = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[2]/div/div/div/div[2]/form/div/div[2]/div[3]/div[1]/input')
submit = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[2]/div/div/div/div[2]/form/div/div[2]/button')

username.send_keys('0988270432')
password.send_keys('@Piwat087')
sleep(3)

submit.send_keys(Keys.ENTER)
sleep(3)

page = 0
shp_product = []

print("Shopee In Progress ...")
while (page <= 1):
    
    driver.get(f'https://shopee.co.th/search?keyword=jewelry&page={page}')
    sleep(3)
    driver.execute_script("document.body.style.zoom='10%'")
    sleep(3)
    
    for item in driver.find_elements(By.CLASS_NAME, 'KMyn8J'):
        try:
            name = item.find_element(By.CLASS_NAME, 'FDn--\+').text
            price = item.find_element(By.CLASS_NAME, 'vioxXd.rVLWG6').text.split()
            quan = item.find_element(By.CLASS_NAME, 'r6HknA.uEPGHT').text.split()
            location = item.find_element(By.CLASS_NAME, 'zGGwiV').text.replace('จังหวัด', '')
            
            price = price[-1].replace('฿', '').replace(',', '')
            quan = quan[1].replace('.', '').replace('พัน', '00')
            shp_product.append((name, price, quan, int(price)*int(quan), location, 'Shopee'))
            
        except NoSuchElementException:
            pass

    page += 1
    print("Page", page, "complete :", len(shp_product), "Product")

print("Shopee Product list:", len(shp_product))
print("Shopee Completed !!!")
driver.close()

shp_products = pd.DataFrame(shp_product, columns = ['name', 'price', 'quantity', 'sales', 'location', 'platforms'])

pd.set_option('display.max_rows', None)
shp_products

In [ ]:
driver = webdriver.Chrome()
page = 1
laz_product = []

print("Lazada In Progress ...")
while (len(laz_product) <= len(shp_product)):
    
    driver.get(f'https://www.lazada.co.th/tag/jewelry/?spm=a2o4m.home.search.1.3c0e7f6dugi2So&q=jewelry&_keyori=ss&from=search_history&sugg=jewelry_0_1&catalog_redirect_tag=true&page={page}')
    sleep(3)
    driver.execute_script("document.body.style.zoom='10%'")
    sleep(3)
    
    for item in driver.find_elements(By.CLASS_NAME, 'buTCk'):
        try:
            name = item.find_element(By.CLASS_NAME, 'RfADt').text
            price = item.find_element(By.CLASS_NAME, 'ooOxS').text.split()
            quan = item.find_element(By.CLASS_NAME, '_1cEkb').text.split()
            location = item.find_element(By.CLASS_NAME, 'oa6ri ').text
            
            price = price[-1].replace('฿', '').replace(',' ,'')
            quan = quan[0].replace(',','').replace('k','000').replace('+','')
            laz_product.append((name, price, quan, float(price)*int(quan), location, 'Lazada'))
            
        except NoSuchElementException:
            pass
    
    print("Page", page, "complete !!!")
    page += 1

print("Lazada Product list:", len(laz_product))
print("Lazada Completed !!!")
driver.close()

laz_products = pd.DataFrame(laz_product,
                            columns=['name', 'price', 'quantity', 'sales', 'location', 'platforms'])
laz_products

In [ ]:
products = pd.concat([shp_products, laz_products])
products.to_csv('ShopeeLazadaJewelryProducts.csv')
products